In [ ]:
@inproceedings{CycleGAN2017,
  title={Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networkss},
  author={Zhu, Jun-Yan and Park, Taesung and Isola, Phillip and Efros, Alexei A},
  booktitle={Computer Vision (ICCV), 2017 IEEE International Conference on},
  year={2017}
}

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

import tensorflow_addons as tfa
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.constraints import Constraint
from tensorflow.keras import backend

import numpy
import random
import glob
from PIL import Image
from numpy import asarray

AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
import json

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
train_horses, train_zebras = [], []
test_horses, test_zebras = [], []

In [ ]:
def turnimagetonumpyA(d,array):
    # load the image and convert into
    # numpy array
    expre = d + '/*_A.jpg'
    jpgFilenamesList = glob.glob(expre)
    jpgFilenamesList = sorted(jpgFilenamesList)
    for name in jpgFilenamesList[:2000]:
        img = Image.open(name)
        # asarray() class is used to convert
        # PIL images into NumPy arrays
        numpydata = asarray(img)
        #  shape
        # print(numpydata.shape)
        numpydata = (numpydata / 127.5) - 1
        array.append(numpydata)

In [ ]:
def turnimagetonumpyB(d,array):
    # load the image and convert into
    # numpy array
    expre = d + '/*_B.jpg'
    # print(expre)
    jpgFilenamesList = glob.glob(expre)
    jpgFilenamesList = sorted(jpgFilenamesList)
    #print(jpgFilenamesList)
    for name in jpgFilenamesList[:2000]:
        img = Image.open(name)
        # asarray() class is used to convert
        # PIL images into NumPy arrays
        numpydata = asarray(img)
        #  shape
        # print(numpydata.shape)
        numpydata = (numpydata / 127.5) - 1
        array.append(numpydata)

In [ ]:
turnimagetonumpyA("../input/cityscapedata/dataset/datasets/cityscapes/trainA",train_horses)      
turnimagetonumpyB("../input/cityscapedata/dataset/datasets/cityscapes/trainB",train_zebras)

In [ ]:
turnimagetonumpyA("../input/cityscapedata/dataset/datasets/cityscapes/testA",test_horses)
turnimagetonumpyB("../input/cityscapedata/dataset/datasets/cityscapes/testB",test_zebras)

In [ ]:
def reshapearray(a):
    a = numpy.array(a)
    a = a.reshape([-1,1,256,256,3])
    print(a.shape)
    return a

In [ ]:
train_horses = reshapearray(train_horses)
train_zebras = reshapearray(train_zebras)

In [ ]:
test_horses = reshapearray(test_horses)
test_zebras = reshapearray(test_zebras)

In [ ]:
plt.subplot(121)
plt.title('Horse')
plt.imshow(train_horses[0][0] * 0.5 + 0.5)

In [ ]:
plt.subplot(121)
plt.title('Zebra')
plt.imshow(train_zebras[0][0] * 0.5 + 0.5)

In [ ]:
# clip model weights to a given hypercube
class ClipConstraint(Constraint):
    # set clip value when initialized
    def __init__(self, clip_value):
        self.clip_value = clip_value
 
    # clip model weights to hypercube
    def __call__(self, weights):
        return backend.clip(weights, -self.clip_value, self.clip_value)
 
    # get the config
    def get_config(self):
        return {'clip_value': self.clip_value}

In [ ]:
# implementation of wasserstein loss
def wasserstein_loss(y_true, y_pred):
    return backend.mean(y_true * y_pred)

In [ ]:
# define the discriminator model
def define_discriminator(image_shape):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    const = ClipConstraint(0.01)
    # source image input
    in_image = Input(shape=image_shape)
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const)(in_image)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    # define model
    model = Model(in_image, patch_out)
    # compile model
    model.compile(loss=wasserstein_loss, optimizer=RMSprop(lr=0.00005))
    return model


In [ ]:
# define image shape
image_shape = (IMG_HEIGHT,IMG_WIDTH,3)
# create the model
model = define_discriminator(image_shape)
# summarize the model
model.summary()

In [ ]:
# generator a resnet block
def resnet_block(n_features, input_layer):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # first layer convolutional layer
    g = Conv2D(n_features, (3,3), padding='same', kernel_initializer=init)(input_layer)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # second convolutional layer
    g = Conv2D(n_features, (3,3), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    # concatenate merge channel-wise with input layer
    g = g + input_layer
    return g

In [ ]:
# define the standalone generator model
def define_generator(image_shape=(256,256,3), n_resnet=6):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=image_shape)
    # c7s1-64
    g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # d128
    g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # d256
    g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # R256
    for _ in range(n_resnet):
        g = resnet_block(256, g)
    # u128
    g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # u64
    g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # c7s1-3
    g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    out_image = Activation('tanh')(g)
    # define model
    model = Model(in_image, out_image)
    return model

In [ ]:
model = define_generator()
# summarize the model
model.summary()

In [ ]:
# define a composite model for updating generators by adversarial and cycle loss
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
    g_model_1.trainable = True
    # mark discriminator as not trainable
    d_model.trainable = False
    # mark other generator model as not trainable
    g_model_2.trainable = False
    # discriminator element
    input_gen = Input(shape=image_shape)
    gen1_out = g_model_1(input_gen)
    output_d = d_model(gen1_out)
    # identity element
    input_id = Input(shape=image_shape)
    output_id = g_model_1(input_id)
    # forward cycle
    output_f = g_model_2(gen1_out)
    # backward cycle
    gen2_out = g_model_2(input_id)
    output_b = g_model_1(gen2_out)
    # define model graph
    model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
    # define optimization algorithm configuration
    opt = Adam(lr=0.0002, beta_1=0.5)
    # compile model with weighting of least squares loss and L1 loss
    model.compile(loss=[wasserstein_loss, 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
    return model

In [ ]:
# input shape
image_shape = (256,256,3)
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

In [ ]:
d_hist, g_hist = list(), list()

In [ ]:
with open("derror.txt", "r") as fp:  
    d_hist = json.load(fp)
with open("gerror.txt", "r") as fp:  
    g_hist = json.load(fp)

In [ ]:
def save_error():
    with open("derror.txt", "w") as fp:  
        json.dump(d_hist, fp)
    with open("gerror.txt", "w") as fp:  
        json.dump(g_hist, fp)
    plt.plot(d_hist, label='crit')
    plt.plot(g_hist, label='gen')
    plt.legend()
    name = "plot_line_plot_loss.png"
    plt.savefig(name)

In [ ]:
def generate_real_samples(dataset, n_samples, patch_shape):
    # choose random instances
    ix = random.randint(0, dataset.shape[0]-1)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = -numpy.ones((n_samples, patch_shape, patch_shape, 1))
    return X, y

In [ ]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
    # generate fake instance
    X = g_model.predict(dataset)
    # create 'fake' class labels (1)
    y = numpy.ones((len(X), patch_shape, patch_shape, 1))
    return X, y

In [ ]:
# update image pool for fake images
def update_image_pool(pool, images, max_size=50):
    selected = list()
    for image in images:
        if len(pool) < max_size:
            # stock the pool
            pool.append(image)
            selected.append(image)
        elif random.random() < 0.5:
            # use image, but don't add it to the pool
            selected.append(image)
        else:
            # replace an existing image and use replaced image
            ix = random.randint(0, max_size-1)
            selected.append(pool[ix])
            pool[ix] = image
    return numpy.asarray(selected)

In [ ]:
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset, n_critic=5):
    # define properties of the training run
    n_epochs, n_batch, = 20, 1
    # determine the output square shape of the discriminator
    n_patch = d_model_A.output_shape[1]
    # unpack dataset
    trainA, trainB = train_horses, train_zebras
    # prepare image pool for fakes
    poolA, poolB = list(), list()
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        dA_loss1,dA_loss2,dB_loss1,dB_loss2=0,0,0,0
        for j in range(n_critic):
            # select a batch of real samples
            X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
            X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
            # generate a batch of fake samples
            X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
            X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
            # update fakes from pool
            X_fakeA = update_image_pool(poolA, X_fakeA)
            X_fakeB = update_image_pool(poolB, X_fakeB)
            dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
            dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
            dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
            dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
        # select a batch of real samples
        X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
        X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
        # generate a batch of fake samples
        X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
        X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
        # update fakes from pool
        X_fakeA = update_image_pool(poolA, X_fakeA)
        X_fakeB = update_image_pool(poolB, X_fakeB)
        # update generator B->A via adversarial and cycle loss
        g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
        # update generator A->B via adversarial and cycle loss
        g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
        d_hist.append(dA_loss2 + dB_loss2)
        g_hist.append(g_loss1 + g_loss2)
        if (i + 1) % 500 == 0:
            ckpt_save_path = ckpt_manager.save()
            print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
            save_error()

In [ ]:
import os
os.chdir(r'/kaggle/input/cityscapedata/results (3)')

In [ ]:
os.chdir(r'/kaggle/working')

In [ ]:
!ls

In [ ]:
checkpoint_path = "pretained"

ckpt = tf.train.Checkpoint(g_model_AtoB=g_model_AtoB,
                           g_model_BtoA=g_model_BtoA,
                           d_model_A=d_model_A,
                           d_model_B=d_model_B,
                           c_model_AtoB=c_model_AtoB,
                           c_model_BtoA=c_model_BtoA)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [ ]:
# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

In [ ]:
ckpt_save_path = ckpt_manager.save()

In [ ]:
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, [])

In [ ]:
print(d_hist)

In [ ]:
print (g_hist)

In [ ]:
save_error()

In [ ]:
def generate_images(model, test_input):
    prediction = model.predict(test_input)
    
    plt.figure(figsize=(12, 12))

    display_list = [test_input[0], prediction[0]]
    title = ['Input Image', 'Predicted Image']

    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()

In [ ]:
for i in range(5):
    X_realA, _ = generate_real_samples(train_zebras, 1, 16)
    generate_images(g_model_BtoA, X_realA)

In [ ]:
for i in range(5):
    X_realA, _ = generate_real_samples(train_horses, 1, 16)
    generate_images(g_model_AtoB, X_realA)

In [ ]:
for i in range(5):
    X_realA, _ = generate_real_samples(test_zebras, 1, 16)
    generate_images(g_model_BtoA, X_realA)

In [ ]:
for i in range(5):
    X_realA, _ = generate_real_samples(test_horses, 1, 16)
    generate_images(g_model_AtoB, X_realA)

In [ ]:
def evaluate(test,answer):
    total = answer.size // 3
    matches = numpy.all(test == answer, axis=-1)
    print(matches.shape)
    print(matches.sum(),total)
    """
    t = {}
    for x in numpy.nditer(answer[:,:,:,:]):
        t.add(x)
    mean_class = []
    for x in t:
        x = numpy.repeat(x,total)
        x = x.reshape(-1,1,256,256,3)
        matches = numpy.all(x == answer, axis=-1)
        class_sum = matches.sum()
        testmatch = numpy.all(x == tes, axis=-1)
        testmatch = numpy.all(testmatch and match, axis=-1)
        mean_class.append(testmatch.sum() / class_sum)
    return matches.sum() / total , mean_class
    """

In [ ]:
realans = test_zebras.reshape([-1,256,256,3])

In [ ]:
print(realans.shape)
print(ans.shape)
print(realans.size)

In [ ]:
ans = (ans + 1) * 127.5
realans = (realans + 1) * 127.5
print(ans)
print(realans[0])

In [ ]:
ans = numpy.around(ans).astype(int)
print(ans[0])
realans = numpy.around(realans).astype(int)
print(realans[0])

In [ ]:
rdict = {}
count = 0
e = realans.reshape([-1,3])
for x in e:
    #print(x)
    x = tuple(x)
    if x not in rdict:
        rdict[x] = count
        count += 1
print(len(rdict))

In [ ]:
evaluate(ans,realans)

In [ ]:
temp = test_horses
temp = temp.reshape([-1,256,256,3])
print(temp)
ans = g_model_AtoB.predict(temp)
print(ans)